In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as skm
#import warnings
#warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (11, 7)

## Import monthly data

In [118]:
# google drive shareable file link

orig_url = 'https://drive.google.com/file/d/1WQ9NARYCNZxWlQUAf4Zn5TxJcnOGjEn3/view?usp=sharing'

# get file id

file_id = orig_url.split('/')[-2]

# create download url

dwn_url='https://drive.google.com/uc?export=download&id=' + file_id

# get raw text inside url

url = requests.get(dwn_url).text

# create a buffer

csv_raw = StringIO(url)

# read from buffer

df = pd.read_csv(csv_raw, index_col=0)

df.reset_index(drop=True, inplace=True)

# create sinop and csinop series

df['date'] = pd.to_datetime(df['date'], format=('%Y-%m-%d')) 

#df.set_index('date', inplace=True)

# cut train period

train = df[(df.date >= '2000-01-01') & (df.date <= '2019-12-01')]

test = df[df.date >= '2019-01-01']

## Flag first full year of data

In [119]:
df['chuva_max_12m'] = df.groupby('cd_estacao')['chuva'].rolling(12).max().reset_index(0,drop=True)

df['flag_chuva_max_12m'] = np.where((~df['chuva_max_12m'].isna()) & (df['chuva_max_12m'] >= 1),
                               1,
                               0)

min_year = df[df['flag_chuva_max_12m']==1].groupby('cd_estacao').agg(min_year=('date','min'))['min_year'].dt.year+1

df = pd.merge(df,min_year,how='left',left_on=df.cd_estacao,right_on=min_year.index)

df.drop(columns='key_0', inplace=True)

## Clustering series

In [122]:
df10 = df[df['min_year']<=2009]

In [143]:
series = []

for cd_estacao in df10.cd_estacao.unique()[0:2]:
    series.append(df10[df10['cd_estacao']==cd_estacao].chuva)

In [145]:
from trendypy.trendy import Trendy
trendy = Trendy(n_clusters=2)
trendy.fit(series)
print(trendy.labels_)

#trendy.predict([[0.9, 2, 3.1, 4]]) # another increasing trend


ModuleNotFoundError: No module named 'trendypy'